In [3]:
# uncomment and run below:
%pip install -qU langchain
%pip install -qU langchain-openai
%pip install tiktoken
%pip install faiss-cpu
%pip install beautifulsoup4
%pip install google-search-results
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import getpass

# Set OPENAI API Key

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [9]:
from langchain_openai import ChatOpenAI

In [10]:
MODEL='gpt-4o-mini'

In [11]:
chat_model = ChatOpenAI(model=MODEL, temperature=0)

In [12]:
output = chat_model.invoke("I am teaching a live-training about LLMs!")
output

AIMessage(content="That sounds exciting! Teaching about large language models (LLMs) can be a great opportunity to share knowledge about cutting-edge technology. Here are some key topics and tips you might consider including in your live training:\n\n### Key Topics to Cover\n\n1. **Introduction to LLMs**:\n   - What are LLMs?\n   - Brief history and evolution of language models (from n-grams to transformers).\n\n2. **How LLMs Work**:\n   - Overview of the transformer architecture.\n   - Explanation of concepts like attention mechanisms, embeddings, and tokenization.\n\n3. **Training LLMs**:\n   - Data collection and preprocessing.\n   - Training processes (supervised vs. unsupervised learning).\n   - Fine-tuning and transfer learning.\n\n4. **Applications of LLMs**:\n   - Natural language processing tasks (text generation, summarization, translation, etc.).\n   - Use cases in various industries (healthcare, finance, customer service).\n\n5. **Ethical Considerations**:\n   - Bias in lan

In [13]:
type(output)

langchain_core.messages.ai.AIMessage

In [14]:
print(output.content)

That sounds exciting! Teaching about large language models (LLMs) can be a great opportunity to share knowledge about cutting-edge technology. Here are some key topics and tips you might consider including in your live training:

### Key Topics to Cover

1. **Introduction to LLMs**:
   - What are LLMs?
   - Brief history and evolution of language models (from n-grams to transformers).

2. **How LLMs Work**:
   - Overview of the transformer architecture.
   - Explanation of concepts like attention mechanisms, embeddings, and tokenization.

3. **Training LLMs**:
   - Data collection and preprocessing.
   - Training processes (supervised vs. unsupervised learning).
   - Fine-tuning and transfer learning.

4. **Applications of LLMs**:
   - Natural language processing tasks (text generation, summarization, translation, etc.).
   - Use cases in various industries (healthcare, finance, customer service).

5. **Ethical Considerations**:
   - Bias in language models and its implications.
   - R

You can predict outputs from both LLMs and ChatModels:

Basic components are:

- Models
- Prompt templates
- Output parsers

In [15]:
from langchain_core.prompts import ChatPromptTemplate

In [16]:
template = "Show me 5 examples of this concept: {concept}"

prompt = ChatPromptTemplate.from_template(template)

prompt.format(concept="animal")

'Human: Show me 5 examples of this concept: animal'

In [17]:
chain = prompt | chat_model

In [18]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [19]:
output = chain.invoke({"concept": "animal"})

In [20]:
output.content

'Sure! Here are five examples of different animals, each representing a unique category:\n\n1. **Mammal**: **Elephant** - The largest land animal, known for its intelligence, social behavior, and strong familial bonds.\n\n2. **Bird**: **Bald Eagle** - A bird of prey known for its impressive wingspan and as a symbol of strength and freedom in the United States.\n\n3. **Reptile**: **Green Iguana** - A large lizard native to Central and South America, known for its vibrant green color and ability to adapt to various environments.\n\n4. **Amphibian**: **Poison Dart Frog** - A small, brightly colored frog found in Central and South America, known for its toxic skin and vibrant colors that serve as a warning to predators.\n\n5. **Fish**: **Clownfish** - A small, colorful fish that lives in sea anemones, known for its symbiotic relationship with the anemone and its popularity due to the animated film "Finding Nemo."\n\nThese examples illustrate the diversity of the animal kingdom across diffe

In [21]:
from IPython.display import Markdown


Markdown(output.content)

Sure! Here are five examples of different animals, each representing a unique category:

1. **Mammal**: **Elephant** - The largest land animal, known for its intelligence, social behavior, and strong familial bonds.

2. **Bird**: **Bald Eagle** - A bird of prey known for its impressive wingspan and as a symbol of strength and freedom in the United States.

3. **Reptile**: **Green Iguana** - A large lizard native to Central and South America, known for its vibrant green color and ability to adapt to various environments.

4. **Amphibian**: **Poison Dart Frog** - A small, brightly colored frog found in Central and South America, known for its toxic skin and vibrant colors that serve as a warning to predators.

5. **Fish**: **Clownfish** - A small, colorful fish that lives in sea anemones, known for its symbiotic relationship with the anemone and its popularity due to the animated film "Finding Nemo."

These examples illustrate the diversity of the animal kingdom across different classes.

You can also use the predict method over a string input:

In [22]:
text = "What would be a good name for a dog that loves to nap??"
chat_model.invoke(text)

AIMessage(content="Here are some cute name ideas for a dog that loves to nap:\n\n1. Snoozer\n2. Naptime\n3. Dreamer\n4. Dozer\n5. Snuggles\n6. Siesta\n7. Zzz\n8. Pillow\n9. Drowse\n10. Napster\n\nChoose one that fits your dog's personality!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 21, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_818c284075', 'finish_reason': 'stop', 'logprobs': None}, id='run-d719f930-17c2-44b3-afad-f5ad8c24b924-0', usage_metadata={'input_tokens': 21, 'output_tokens': 74, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [23]:
from langchain_core.prompts  import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'Human: What is a good dog name for a dog that loves to sleeping?'

In [24]:
chain = prompt | chat_model

chain.invoke({'activity': 'sleeping'})

AIMessage(content="Here are some cute and fitting names for a dog that loves to sleep:\n\n1. **Snoozer**\n2. **Napster**\n3. **Dozer**\n4. **Slumber**\n5. **Dreamer**\n6. **Pillow**\n7. **Cuddles**\n8. **Naptime**\n9. **Resty**\n10. **Zzz**\n\nChoose one that resonates with your dog's personality!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 21, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9f7ab7a-d302-4706-8e45-e063ba228f76-0', usage_metadata={'input_tokens': 21, 'output_tokens': 88, 'total_tokens': 109, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [26]:
chain.invoke({"activity": "Landscapes"})

AIMessage(content='A great name for a dog that loves landscapes could be "Scenic." Other options might include "Vista," "Meadow," "Canyon," or "Willow." These names evoke the beauty of nature and the outdoors, reflecting your dog\'s love for landscapes!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-397bd2a2-0c2b-4c3e-aec6-42b8f0b4ca23-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [28]:
llm = ChatOpenAI(model=MODEL, temperature=0.0)
prompt = ChatPromptTemplate.from_template("""
Write 5 concepts that are fundamental to learn about {topic}.
                                          """)
chain = prompt | llm | output_parser
chain.invoke({"topic": "Artificial Neural Networks"})

"Certainly! Here are five fundamental concepts that are essential to understand when learning about Artificial Neural Networks (ANNs):\n\n1. **Neurons and Activation Functions**:\n   - At the core of ANNs are artificial neurons, which are inspired by biological neurons. Each neuron receives inputs, processes them, and produces an output. The output is typically passed through an activation function, which introduces non-linearity into the model. Common activation functions include Sigmoid, ReLU (Rectified Linear Unit), and Tanh. Understanding how these functions work and their impact on the network's performance is crucial.\n\n2. **Network Architecture**:\n   - The architecture of an ANN refers to its structure, including the number of layers (input, hidden, and output layers) and the number of neurons in each layer. Different architectures (e.g., feedforward networks, convolutional neural networks, recurrent neural networks) are suited for different types of tasks. Learning how to des

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [29]:
chain = prompt | llm

chain.invoke({"topic": "sleep"})

AIMessage(content='Certainly! Here are five fundamental concepts to understand about sleep:\n\n1. **Sleep Stages and Cycles**: Sleep is divided into several stages, primarily categorized into Non-Rapid Eye Movement (NREM) and Rapid Eye Movement (REM) sleep. A typical sleep cycle lasts about 90 minutes and includes multiple stages, each serving different functions, such as physical restoration, memory consolidation, and emotional regulation.\n\n2. **Circadian Rhythms**: These are the natural, internal processes that regulate the sleep-wake cycle and repeat roughly every 24 hours. Circadian rhythms are influenced by external cues like light and temperature, and they play a crucial role in determining sleep patterns, alertness, and overall health.\n\n3. **Sleep Hygiene**: This refers to a set of practices and habits that promote good quality sleep. Key components include maintaining a consistent sleep schedule, creating a comfortable sleep environment, limiting exposure to screens before 

In [30]:
from langchain_ollama import ChatOllama

In [31]:
# ollama pull llama3 in the terminal
llm = ChatOllama(model="llama3.2")

In [32]:
type(llm)

langchain_ollama.chat_models.ChatOllama

In [33]:
chain = prompt | llm | output_parser

chain.invoke({"topic": "neuroscience of sleep"})

'Here are five fundamental concepts to learn about neuroscience of sleep:\n\n1. **Sleep Cycles**: Sleep is composed of multiple stages, including light and deep sleep. Understanding the different stages of sleep is crucial in understanding how the brain functions during rest. A full sleep cycle typically lasts around 90-120 minutes and consists of three stages:\n\t* Stage 1: Transition from wakefulness to sleep\n\t* Stage 2: Light sleep with slow brain waves\n\t* Stage 3: Deep sleep with slow delta brain waves\n2. **Neurotransmitters and Hormones**: Neurotransmitters such as GABA, glutamate, and serotonin play a crucial role in regulating sleep-wake cycles. Hormones like melatonin, cortisol, and adenosine also influence sleep patterns. Understanding the roles of these chemicals is essential to grasping how the brain controls sleep.\n3. **Brain Regions Involved in Sleep**: Different brain regions are active during different stages of sleep. For example:\n\t* The hippocampus, involved in

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

# Our First LangChain App

See `./1.1-langchain-app.py`

# LangChain Exercise

Let's create a simple chain for summarization of content. 

Your chain should:

- A prompt template with one or more variables
- A model like ChatGPT or other (you can use local models if you'd like, I recommend `ChatOllama` for that!)
- Optional: use output parsing or just fetch the string output at the end!

## Example Answer

Let's make use of the `ChatPromptTemplate` to abstract away the following pieces of the prompt: 
- `content` - the text content to be summarized  
- `summary_format` - the format in which we want the summary to be presented (like bullet points and so on).

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this: {content}. The output should be in the following format: {summary_format}.")

# We can look at a simple example to illustrate what that prompt is doing
prompt.format(content="This is a test.", summary_format="One word summary")

'Human: Summarize this: This is a test.. The output should be in the following format: One word summary.'

Ok, now that we have our prompt template done, let's load the llm and create a nice chain to put everything together. 

In [35]:
from langchain_openai import ChatOpenAI

llm_chat =  ChatOpenAI()
chain = prompt | llm_chat # This is the Pipe symbol! from LCEL that connect model to prompt!

Now, that we have our chain we can run some tests. The cool thing about working with LLMs is that you can use them to create examples for simple tests like this (avoiding the annoynace of searching online for some piece of text, copying and pasting etc...). So, let's generate a few examples of tests below:

In [36]:
num_examples = 3
examples = []
for i in range(num_examples):
    examples.append(llm_chat.invoke("Create a piece of text with 2 paragraphs about a random topic regarding human-machine interaction."))

examples

[AIMessage(content='One of the most fascinating aspects of human-machine interaction is the concept of artificial intelligence and its ability to mimic human behavior. From chatbots that can hold conversations with users to voice assistants that can schedule appointments and answer questions, AI has become increasingly integrated into our daily lives. This technology has the potential to revolutionize industries such as healthcare, transportation, and finance, but also raises concerns about privacy, security, and the ethical implications of relying on machines to make decisions for us.\n\nAnother important aspect of human-machine interaction is the idea of augmented reality and virtual reality, which allow users to immerse themselves in digital environments and interact with virtual objects. These technologies have the potential to enhance education, entertainment, and communication, but also blur the lines between reality and fiction. As we continue to develop more advanced human-mach

Nice! Now that we have our examples, let's run our chain on them and check out the results.

In [37]:
summary_format = "bullet points"

outputs = []
for ex in examples:
    outputs.append(chain.invoke({"content": ex, "summary_format": summary_format}))

# Let's display one example output
outputs[0]

AIMessage(content='- Artificial intelligence is a fascinating aspect of human-machine interaction that mimics human behavior\n- AI is integrated into daily life through chatbots, voice assistants, and has the potential to revolutionize industries\n- Concerns arise about privacy, security, and ethical implications of relying on machines for decision-making\n- Augmented reality and virtual reality enhance education, entertainment, and communication\n- These technologies blur the lines between reality and fiction\n- It is crucial to consider the impact of advanced human-machine interfaces on cognitive abilities, social interactions, and overall well-being\n- The future of human-machine interaction depends on balancing the benefits of technology with potential risks and challenges', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 460, 'total_tokens': 590, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio

Great! So it seems our chain worked and we generated some summaries! Let's visualize all the summaries generated in a neat way.

In [39]:
from IPython.display import Markdown

for i in range(num_examples):
    display(Markdown(f"Output {i} \n {outputs[i].content}"))
# Markdown(f"**Input**: {examples[0]}\n\n**Output**: {outputs[0]}")

Output 0 
 - Artificial intelligence is a fascinating aspect of human-machine interaction that mimics human behavior
- AI is integrated into daily life through chatbots, voice assistants, and has the potential to revolutionize industries
- Concerns arise about privacy, security, and ethical implications of relying on machines for decision-making
- Augmented reality and virtual reality enhance education, entertainment, and communication
- These technologies blur the lines between reality and fiction
- It is crucial to consider the impact of advanced human-machine interfaces on cognitive abilities, social interactions, and overall well-being
- The future of human-machine interaction depends on balancing the benefits of technology with potential risks and challenges

Output 1 
 - The relationship between humans and machines is becoming more complex as technology advances.
- The focus in human-machine interaction is on making interactions seamless and intuitive for users.
- Designing easy-to-navigate interfaces and ensuring machines can interpret human input are key aspects.
- Ethical considerations arise when integrating technology into daily lives, such as concerns about job impact, privacy, and security.
- Designers and developers must consider ethical implications and aim to create technology that enhances human capabilities.
- Balancing human needs with machine capabilities is crucial to ensuring beneficial and empowering human-machine interaction for all users.

Output 2 
 - Artificial intelligence can understand and predict human behavior by analyzing data and interpreting patterns
- This ability has the potential to revolutionize industries like marketing, healthcare, and transportation
- Raises ethical concerns about privacy and autonomy as machines can manipulate human behavior based on predictions
- Researchers are exploring emotional intelligence in machines to recognize and respond to human emotions
- Developing emotional intelligence in machines could lead to more personalized interactions and a seamless user experience
- Ensuring machines can accurately interpret and respond to complex human emotions is a challenge with significant implications for interactions and individual well-being

Great! Our summaries worked, and we were able to apply a given summary format to all of them.

LangChain is an extremely powerful library to work with abstractions like these and throughout this course we hope to give you a gliimpse of the cool stuff you can build with it.

# Introduction to LangChain Expression Language ([LCEL](https://python.langchain.com/docs/get_started/introduction))

LCEL is a declarative way to compose chains of components. 

What does that mean? Means its an easy way to put useful building blocks together.


Here's quick summary of the LangChain Expression Language (LCEL) page:

- LCEL Basics: Simplifies building complex chains from basic components using a unified interface and composition primitives.

- Unified Interface: Every LCEL object implements the Runnable interface, supporting common invocation methods like invoke, batch, stream, ainvoke, and more.

- Composition Primitives: LCEL provides tools for composing chains, parallelizing components, adding fallbacks, and dynamically configuring internal chain elements.

- Model Flexibility: LCEL allows for easy switching between different models and providers (like OpenAI or Anthropic), and runtime configurability of chat models or LLMs.

- Advanced Features: LCEL features things like logging intermediate results with LangSmith integration and adding fallback logic for enhanced reliability.

Ok, cool but what is a component?

A component is something that implements the `Runnable` protocol.


Ok....and what is that?

It's an object with some nice desirable features like:

- input and output schemas (describe what that object takes as input and the structure of its output)

Some nice methods are:

- `invoke` [ainvoke]
- `stream` [astream]
- `batch` [abatch]


Below is a list of common i/o types for each component:

![](./assets-resources/components_input_type_output_type.png)

[source for the image](https://python.langchain.com/docs/expression_language/interface)

In [40]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

output = model.invoke("hi")

In [41]:
type(output)

langchain_core.messages.ai.AIMessage

In [42]:
model.input_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [43]:
model.output_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

Let's look at a simple example.

In [44]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm_chat = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(("Translate this {word} into {language}"))
output_parser = StrOutputParser()

chain = prompt | llm_chat | output_parser

chain.invoke({"word": "responsibility", "language": "Italian"})

'questa responsabilità'

Ok nice! So we put everything together using this [pipe](https://en.wikipedia.org/wiki/Pipeline_(Unix)) `|` symbol (or [unix pipe operator](https://en.wikipedia.org/wiki/Pipeline_(Unix)) if you want to get fancy) That's the power of the LCEL language, putting different components together through a simple interface.

[source](https://python.langchain.com/docs/expression_language/get_started#:~:text=its%20cone-fidence!%22-,4.%20entire%20pipeline)

So what is happening is:

- We pass in user input on the desired concept as {"concept": "machine learning"}
- The prompt component takes the user input, which is then used to construct a `PromptValue` after using the `concept` to construct the `prompt`.
- The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a `ChatMessage` object.
- Finally, the `output_parser` component takes in a `ChatMessage`, and transforms this into a Python string, which is returned from the invoke method.

```mermaid
graph LR;
    A[Input concept=machine learning] --dict--> B[Prompt Template]
    B --PromptValue--> C[ChatModel]
    C --ChatMessage--> D[StrOutputParser]
    D --string--> E[Output]
```

Here's a bullet point summary of the key features and benefits of LangChain Expression Language (LCEL):

Declarative Composing: LCEL allows for easy composition of chains, ranging from simple "prompt + LLM" chains to complex ones with hundreds of steps.

Streaming Support: LCEL offers optimal time-to-first-token, enabling streaming of tokens from an LLM to a streaming output parser for quick, incremental output.

Async Support: Chains built with LCEL can be used both synchronously (e.g., in Jupyter notebooks for prototyping) and asynchronously (e.g., in a LangServe server), maintaining consistent code for prototypes and production.

Optimized Parallel Execution: LCEL automatically executes parallel steps in a chain (like fetching documents from multiple retrievers) in both sync and async interfaces, reducing latency.

Retries and Fallbacks: Users can configure retries and fallbacks for any part of the LCEL chain, enhancing reliability at scale. Streaming support for these features is in development.

Access to Intermediate Results: LCEL allows access to intermediate step results, useful for user updates or debugging. This feature includes streaming intermediate results and is available on all LangServe servers.

Input and Output Schemas: LCEL chains come with Pydantic and JSONSchema schemas, inferred from the chain's structure, which aid in validating inputs and outputs. This is a core part of LangServe.

Seamless LangSmith Tracing Integration: As chains become more complex, LCEL provides automatic logging of all steps to LangSmith for enhanced observability and debuggability.

Seamless LangServe Deployment Integration: LCEL chains can be easily deployed using LangServe, facilitating smoother deployment processes.

These features highlight LCEL's versatility and efficiency in both development and production environments, making it a powerful tool for creating and managing complex language chains.

Now, let's look into the `Runnable` components in more detail. 

`RunnableLambda`

In [45]:
from langchain.schema.runnable import RunnableLambda


# Create a RunnableLambda and invoke it
runnable = RunnableLambda(lambda x: x*23)
output = runnable.invoke(5)
print(output)  # Output: 115

115


[`RunnableSequence`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnablesequence%20invokes%20a%20series%20of%20runnables%20sequentially%2C%20with%20one%20runnable%E2%80%99s%20output%20serving%20as%20the%20next%E2%80%99s%20input.%20construct%20using%20the%20%7C%20operator%20or%20by%20passing%20a%20list%20of%20runnables%20to%20runnablesequence.)

RunnableSequence invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to RunnableSequence.

In [46]:
from langchain_core.runnables import RunnableSequence
# Suppose we have a list of numbers
data = [1, 2, 3, 4, 5]

# Create RunnableLambdas
preprocess_runnable = RunnableLambda(lambda x: [i+1 for i in x])
apply_model_runnable = RunnableLambda(lambda x: x*23)
postprocess_runnable = RunnableLambda(lambda x: sum(x))

# Create a RunnableSequence and invoke it
sequence = preprocess_runnable | apply_model_runnable | postprocess_runnable
# runnable_sequence = RunnableSequence(sequence)
output = sequence.invoke(data)

output

460

In [12]:
# [1,2,3,4,5]

# # Runnable 1
# [2,3,4,5,6]

# # Runnable 2 
# [2,3,4,5,6] * 23

In [47]:
print(output)
# or
runnable_sequence = RunnableSequence(first=preprocess_runnable, middle=[apply_model_runnable], last=postprocess_runnable)
runnable_sequence.invoke(data)

460


460

[`Runnableparallel`]([`Runnableparallel`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)

RunnableParallel invokes runnables concurrently, providing the same input to each. Construct it using a dict literal within a sequence or by passing a dict to RunnableParallel.

In [48]:
import langchain
print(langchain.__version__)

0.3.3


In [49]:
from langchain.schema.runnable import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

[4, 6, 8]

In [50]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

In [51]:
from langchain_core.runnables import RunnableParallel

sequence_with_runnable_parallel = RunnableParallel(
    multiply=RunnableLambda(lambda x: x * 2),
    add=RunnableLambda(lambda x: x + 2))

sequence_with_runnable_parallel.invoke(1) # [2, 5]

{'multiply': 2, 'add': 3}

When you build chains, what you're doing is building a type of Runnable!

That could be a RunnableSequence or a RunnableParallel  

In [52]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI()
prompt = PromptTemplate.from_template("Summarize this {text}")
prompt2 = PromptTemplate.from_template("Explain this {text}")


chain1 = prompt | llm

chain2 = prompt2 | llm

runnable_parallel = RunnableParallel(summary=chain1, explanation=chain2)

runnable_parallel.invoke({"text": "Single responsibility principle in programming"})

{'summary': AIMessage(content='The Single Responsibility Principle in programming states that a class should have only one reason to change, meaning it should only have one responsibility or job to do. This helps to keep code organized, maintainable, and easier to understand. It promotes modular and flexible code design by separating concerns and preventing classes from becoming too complex or tightly coupled.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 16, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f6deea2f-839d-4f70-9af8-ac5ffa4b8c34-0', usage_metadata={'input_tokens': 16, 'output_tokens': 66, 'total_tokens': 82, 'input

In [53]:
type(runnable_parallel)

langchain_core.runnables.base.RunnableParallel

[RunnableParallel can be useful for manipulating the output of one Runnable to match the input format of the next Runnable in a sequence.](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=RunnableParallel%20can%20be,the%0A%E2%80%9Cquestion%E2%80%9D%20key.)

RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

([see it in docs here](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=runnableparallel%20(aka.%20runnablemap)%20makes%20it%20easy%20to%20execute%20multiple%20runnables%20in%20parallel%2C%20and%20to%20return%20the%20output%20of%20these%20runnables%20as%20a%20map.))

In [56]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship any longer!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a217e92d-a12c-4d64-bbfd-276da18e4d4a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'poem': AIMessage(content='In the dark forest, a bear roams free,\nWith strength and grace, a wild beauty to see.', additional_kwargs={'refusal': None}, response_metadata={'token_

RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel. For example, we can see our earlier joke_chain, poem_chain and map_chain all have about the same runtime, even though map_chain executes both of the other two.

In [57]:
%%timeit

joke_chain.invoke({"topic": "bear"})

572 ms ± 78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%%timeit

poem_chain.invoke({"topic": "bear"})

477 ms ± 26.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%%timeit

joke_chain.invoke({"topic": "bear"})

poem_chain.invoke({"topic": "bear"})

1.11 s ± 88.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%%timeit

map_chain.invoke({"topic": "bear"})

528 ms ± 35.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[RunnablePassthrough](https://python.langchain.com/docs/expression_language/how_to/passthrough#:~:text=RunnablePassthrough%20allows%20to,pass%20it%20through.)

Allows you to pass inputs unchanged or with addition of new keys.

Usually you would connect this with `RunnableParallel` to assign data to new key in the map.

If you just call it, it will take the input and pass it along.

In [61]:
from langchain.schema.runnable import RunnablePassthrough

# Create a RunnablePassthrough
passthrough = RunnablePassthrough()
passthrough.invoke(5) # 5

5

But if you call it with the `.assign(...)` mthod, then it will take the input, and add extra arguments passed to the assign function. 

The key being added has to be a lambda function.

In [62]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# Passthrough with assignment, adds an extra key-value pair
runnable = RunnablePassthrough.assign(extra_key_add_5=lambda x:  x["num"]+5)
input_data = {"num": 1}
result = runnable.invoke(input_data)
# Output: {'num': 1, 'extra_value': 42}
result

{'num': 1, 'extra_key_add_5': 6}

In combination with RunnableParallel:

In [63]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}

{'origin': 1, 'modified': 2}

In [64]:
# modified example from langchain docs: https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html?highlight=runnablepassthrough#langchain_core.runnables.passthrough.RunnablePassthrough 
# or for an "LLM" example:

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] + " (applied the parsing logic)" # Parsing logic
}

chain.invoke('hello') # {'original': 'completion', 'parsed': 'noitelpmoc'}

{'original': 'completion', 'parsed': 'noitelpmoc (applied the parsing logic)'}

Ok so we have these 4 types of main objects:

- `RunnableLambda`
- `RunnableSequence`
- `RunnablePassthrough`
- `RunnableParallel`

combined they make up kind of the core building block system of langchain allow you to create complex inner logics powered by llms. 

Let's create some fun examples below:

In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough

def fun_llm(prompt: str):
    return ChatOpenAI().invoke(f"Make this funny: {prompt}")


def corporate_llm(prompt: str):
    return ChatOpenAI().invoke(f"Make this topic a subject of a very corporate email: {prompt}")

fun_chain = RunnableLambda(fun_llm) | {
    "original-text-input": RunnablePassthrough(), 
    "make-it-corporate": lambda x: corporate_llm(x)
}


fun_chain.invoke("gorillas")

{'original-text-input': AIMessage(content='Why did the gorilla bring a ladder to the party? \nBecause he heard the drinks were on the house!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 13, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e2ee0ba-130f-40ab-bbca-32ff4ba3cf74-0'),
 'make-it-corporate': AIMessage(content='Subject: Hilarious Joke to Lighten Up Your Day! 🤣🎉\n\nDear Team,\n\nI hope this email finds you well. I wanted to share a little humor to brighten up your day.\n\n"Why did the gorilla bring a ladder to the party? Because he heard the drinks were on the house!"\n\nI hope this joke brings a smile to your face and adds a touch of laugh

In [66]:
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
  )

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

{'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

Breaking down whats happening:

We create a `runnable` object, which is an instance of the `RunableSequence` object. This chain takes in as input a simple string: `hello`, and what it does is it 
simulates passing that same string to 2 `different` llms (which in this case are the same) and applies the logic of each of these llms as well as a third logic described by the new extra key added by the RunnablePassthrough.assign() method, that takes as input the output of the `llm1` and `llm2` as input to combine it through the lambda function associated with the new extra key created: `total_chars`. 

Let's make this example a bit more interesting by applying some research workflow vibe to it:

In [70]:
# let's create a couple of llms to do different things with some piece of content.
# In this case let's use different llms to create summarization levels for some text.
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def llm_summarization_level1(prompt: str):
    return ChatOpenAI().invoke(f"Summarize this text: {prompt}")

def llm_summarization_level2(prompt: str):
    return ChatOpenAI().invoke(f"Summarize this text in bullet points: {prompt}")

def llm_summarization_level3(prompt: str):
    return ChatOpenAI().invoke(f"Summarize this text into one short paragraph: {prompt}")


# now let's create a chain that will run all of these llms in parallel and then return the results

runnable = {
    'level1':  llm_summarization_level1,
    'level2':  llm_summarization_level2,
    'level3':  llm_summarization_level3,
} | RunnablePassthrough.assign(prompt_level4=lambda x: f"Combine these 3 summarizations\
    into one combining all their \n \
    useful features. \
    Level1: {x['level1']}.\n \
    Level2: {x['level2']}.\n\
    Level3: {x['level3']}"
                       ) | RunnablePassthrough.assign(output=RunnableLambda(lambda x: ChatOpenAI().invoke(x['prompt_level4']))) 
# For the assign method with RunningPasshthrough, we'll combine the summarizations into a fourth one.

output = runnable.invoke(
    """
    The code you provided is from the nest_asyncio library, which is designed to patch Python's asyncio library to allow nested usage of the asyncio event loop. To understand this code, it's crucial first to grasp what an event loop is in the context of asynchronous programming.
What is an Event Loop?
An event loop in programming, particularly in asynchronous programming, is a central control structure that manages and dispatches events or messages in a program. In the context of Python's asyncio library, the event loop is a core feature that runs asynchronous tasks and callbacks, handles network IO operations, and manages subprocesses. It is essentially the heart of the asyncio module, enabling asynchronous programming by juggling and scheduling the execution of various tasks.
Key Concepts in the nest_asyncio Code
Patching asyncio: The code modifies (patches) certain parts of the asyncio library. This is done to change the default behavior of asyncio, particularly to support nested event loops, which are not allowed in standard asyncio.
Reentrancy in Event Loops: The primary function of nest_asyncio is to make the asyncio event loop reentrant. In computing, reentrancy refers to the ability of a function to be paused in the middle of execution and safely called again ("re-entered") before its previous executions are complete. This is not normally supported by the asyncio event loops, as they are designed to prevent re-entry (or nesting) to avoid complex problems and unexpected behavior.
Modifying Loop Behavior: The code alters the behavior of the event loop methods like run_forever, run_until_complete, and the internal _run_once. These modifications allow the event loop to pause and resume (re-enter) gracefully, facilitating nesting.
Context Managers: The code uses context managers (manage_run and manage_asyncgens) to properly manage the state of the event loop during entry and exit of asynchronous contexts, which is crucial for handling nested loops correctly.
Task Patching: It also patches the Task class of asyncio to modify its step function. This is necessary to ensure that the tasks (units of work scheduled by the event loop) behave correctly in a nested loop scenario.
Tornado Patching: If the Tornado library (an asynchronous networking library) is used, nest_asyncio makes Tornado aware of the Python asyncio Future, ensuring compatibility.
Understanding the Event Loop in Async Programming
In asynchronous programming, especially in Python's asyncio, the event loop is pivotal. It allows the execution of multiple tasks seemingly in parallel by switching between them. This switching is non-blocking, meaning the program can handle other tasks while waiting for some IO operation to complete, thereby increasing efficiency and responsiveness.

The nest_asyncio library's primary role is to tweak the asyncio's event loop to support nested operation, which is particularly useful in scenarios like running an asyncio event loop inside another asyncio event loop, something that standard asyncio does not support by default.

For more details on asynchronous programming and event loops in Python, the Python asyncio documentation is a comprehensive resource.
"""
)

In [73]:
from IPython.display import Markdown

Markdown(output['level1'].content)

The text discusses the nest_asyncio library, which patches Python's asyncio library to allow nested usage of the asyncio event loop. It explains the concept of an event loop in asynchronous programming and highlights key concepts in the nest_asyncio code, such as patching asyncio, enabling reentrancy in event loops, modifying loop behavior, using context managers, and task patching. The library's primary role is to support nested operation of the asyncio event loop, which is not supported by standard asyncio. Async programming relies heavily on event loops to manage and execute tasks efficiently.

In [74]:
Markdown(output['level2'].content)

- The code provided is from the nest_asyncio library, which patches Python's asyncio library to allow nested usage of the event loop
- An event loop in programming manages and dispatches events or messages in a program
- Key concepts in the nest_asyncio code include patching asyncio, making the event loop reentrant, modifying loop behavior, using context managers, task patching, and Tornado patching
- The event loop in asynchronous programming allows for executing multiple tasks seemingly in parallel by switching between them
- The nest_asyncio library tweaks asyncio's event loop to support nested operation, which is useful for running an asyncio event loop inside another asyncio event loop.

In [76]:
Markdown(output['level3'].content)

The nest_asyncio library modifies Python's asyncio library to enable nested usage of the asyncio event loop. It alters the behavior of the event loop to support reentrancy, modifies loop methods, uses context managers, and patches the Task class to handle nested loops correctly. This allows for efficient execution of multiple tasks in parallel in asynchronous programming. The library also ensures compatibility with the Tornado library and is crucial for understanding event loops in async programming.

A simple research assistant, heavily based on this implementation by Harrison Chase:
- https://gist.github.com/hwchase17/69a8cdef9b01760c244324339ab64f0c

In [91]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import requests
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import os
from langchain.tools import tool
from serpapi import GoogleSearch
import json

RESULTS_PER_QUESTION = 1

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}

def web_search(query: str) -> str:
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
    results = search.get_dict()["organic_results"]
    urls = [r["link"] for r in results]
    
    return urls


def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"


def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

In [92]:
SUMMARY_TEMPLATE = """{text} 
-----------
Using the above text, answer in short the following question: 
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""  # noqa: E501
SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)


url = "https://python.langchain.com/docs/concepts/"

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

At this stage, even though this chain requires two keys in the input dictionary, that's not reflected in the input_schema() of the chain, because in it only goes the variables contained in the ChatPromptTemplate(). 

Interestingly enough, the `text` variable which is in the ChatPromptTemplate, is not in the schema because its already part of the chain itself (therefore not being necessary in the input_schema() I guess).

In [93]:
# At this
# let's test this chain on some url
scrape_and_summarize_chain.invoke({"question": "What are some important concepts in langchain?", "url": url})

'URL: https://python.langchain.com/docs/concepts/\n\nSUMMARY: Some important concepts in LangChain include:\n\n1. **Chat Models**: LLMs exposed via a chat API that process sequences of messages.\n2. **Messages**: The unit of communication in chat models, representing model input and output.\n3. **Chat History**: A sequence of messages that represent a conversation.\n4. **Tools**: Functions with associated schemas defining their name, description, and accepted arguments.\n5. **Tool Calling**: A chat model API that accepts tool schemas along with messages as input.\n6. **Structured Output**: A technique for chat models to respond in a structured format, such as JSON.\n7. **Memory**: Information about a conversation that is persisted for future use.\n8. **Multimodality**: The ability to work with various forms of data, including text, audio, images, and video.\n9. **Runnable Interface**: The base abstraction for many LangChain components.\n10. **Streaming**: APIs for surfacing results as 

In [94]:
web_search_chain = RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"]) # urls will be the output of the web_search function()
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()

Let's now test this web search chain.

In [95]:
web_search_chain.invoke({"question": "Look up tool calling in langchain"})

['URL: https://python.langchain.com/docs/concepts/tool_calling/\n\nSUMMARY: Tool calling in LangChain refers to the ability of models to interact with external systems, such as databases or APIs, by using structured input schemas. Key concepts include:\n\n1. **Tool Creation**: Tools are created using the `@tool` decorator, which associates a function with its schema.\n2. **Tool Binding**: Tools must be connected to a model that supports tool calling, which allows the model to recognize the tool and its input requirements.\n3. **Tool Calling**: The model can decide when to call a tool, ensuring that its response adheres to the tool\'s input schema.\n4. **Tool Execution**: The tool is executed using the arguments provided by the model.\n\nThe recommended workflow involves creating tools, binding them to a model with the `.bind_tools()` method, and invoking the model with user input. If a tool call is made, the model\'s response will include the arguments for the tool.\n\nExample of tool 

In [96]:
SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 google search queries to search online that form an "
            "objective opinion from the following: {question}\n"
            "You must respond with a list of strings in the following format: "
            '["query 1", "query 2", "query 3"].',
        ),
    ]
)

search_question_chain = SEARCH_PROMPT | ChatOpenAI(temperature=0) | StrOutputParser() | json.loads

In [97]:
search_question_chain.invoke({"question": "What are the most useful langchain utilities for research?"})

['best langchain utilities for research',
 'top langchain tools for academic research',
 'langchain software for academic studies']

In [98]:
full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

In [99]:
full_research_chain.invoke({"question": "What is langchain?"})

[['URL: https://www.langchain.com/\n\nSUMMARY: LangChain is a composable framework designed for building applications that utilize large language models (LLMs). It offers a suite of products, including LangSmith for debugging, testing, and monitoring LLM applications, and LangGraph for orchestrating agent-driven workflows. LangChain supports developers throughout the LLM application lifecycle, enabling the creation of context-aware and reasoning applications that leverage company data and APIs. The platform has achieved over 15 million monthly downloads, powers more than 100,000 apps, has over 100,000 GitHub stars, and boasts a community of over 4,000 contributors. It is utilized by teams across various industries to enhance operational efficiency, discovery, personalization, and product quality.',
  'URL: https://aws.amazon.com/what-is/langchain/\n\nSUMMARY: LangChain is an open-source framework designed for building applications based on large language models (LLMs). It provides tool

In [101]:
WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501


# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""  # noqa: E501

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)


chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | prompt | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()


`RunnableMap`

> Let's modify this example for something that uses the updated version of the pydantic library.

In [102]:
# this example requires pydantic==
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


llm_chat = ChatOpenAI()

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("Where did harrison work?")

template = "Answer the question based on this context: {context}\n\nQuestion: {question}"
prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
    
}) | prompt | llm_chat | output_parser



chain.invoke({"question": "What did harrison do?"})

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_32739/2928158330.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Where did harrison work?")
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Harrison worked at Kensho.'

If we inspect just the `RunnableMap`:

In [103]:
from langchain.schema.runnable import RunnablePassthrough
inputs_with_lambda = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
})

inputs_with_lambda.invoke({"question": "What did harrison do?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'What did harrison do?'}

`RunnableMap` is an alias for `RunnableParallel`.

In [104]:
inputs_with_runnable_passthrough = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
})

inputs_with_runnable_passthrough.invoke({"question": "What did harrison do?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'What did harrison do?'}

You get the same thing!

You can also use tools with runnables in some neat and interesting ways. Let's see that in the next notebook (2.2)

We see that what the `RunnableMap` is doing, is create the right inputs for the downstream of this chain which in this case is seeting the `context` key with a value of a list of `Document` objects, and the question key with itself.
(which now we can probably change to something simpler with the `RunnablePassthrough` for example).